In [1]:
import sys
sys.path.append("/home/rkube/repos/frnn-loader")

In [2]:
import torch
import torch.utils.data as data
import h5py
import numpy as np

import tempfile

In [3]:

from frnn_loader.backends.fetchers import fetcher_d3d_v1
from frnn_loader.primitives.resamplers import resampler_causal
from frnn_loader.backends.backend_hdf5 import backend_hdf5
from frnn_loader.primitives.signal import signal_0d
from frnn_loader.loaders.frnn_dataset_disk import shot_dataset_disk
from frnn_loader.utils.errors import BadDownloadError

In [4]:
root = "/home/rkube/tmp/tmpdq4efpr1"
basedir = "/projects/FRNN/shot_lists"

In [5]:
my_resampler = resampler_causal(0.0, 2e3, 1e0)

# Instantiate a file backend
my_backend_file = backend_hdf5("/home/rkube/datasets/frnn/")
my_fetcher = fetcher_d3d_v1()
root = root

signal_fs07 = signal_0d("fs07")
signal_q95 = signal_0d("q95")
signal_pinj = signal_0d("bmspinj")

ds = shot_dataset_disk(167475, 
                       predictors=[signal_fs07, signal_q95], 
                       resampler=my_resampler, 
                       backend_file=my_backend_file, 
                       fetcher=my_fetcher, 
                       root=root,
                       download=True,
                       dtype=torch.float32)

In [13]:
print(ds[0])
print(ds[200])
print(ds[400])

tensor([[-7.9470e+10,  1.2168e+01]])
tensor([[6.2916e+11, 1.1499e+01]])
tensor([[2.0464e+12, 8.4916e+00]])


In [11]:
ds[torch.tensor([0, 200, 400])]

idx is a tensor


tensor([[-7.9470e+10,  1.2168e+01],
        [ 6.2916e+11,  1.1499e+01],
        [ 2.0464e+12,  8.4916e+00]])

In [12]:
ds[torch.tensor([0, 400, 200])]

idx is a tensor


tensor([[-7.9470e+10,  1.2168e+01],
        [ 2.0464e+12,  8.4916e+00],
        [ 6.2916e+11,  1.1499e+01]])

In [ ]:
o = torch.zeros((10, 3))

o[:, 1:2] = torch.tensor(np.random.randn(10, 1))


In [ ]:
np.random.randn(10, 2)

In [ ]:
class MyDataset(data.Dataset):
    def __init__(self):
        self.resampler = range(100)
        self.data = torch.rand(100, 5)

    def __len__(self) -> int:
        print("calling __len__")
        return len(self.resampler)

    def __getitem__(self, i) -> torch.Tensor:
        print(f"__getitem__: {i}")
        return self.data[i]

In [ ]:
dataset = MyDataset()
sampler = data.BatchSampler(data.RandomSampler(dataset), 4, False)
loader  = data.DataLoader(dataset, sampler=sampler, batch_size=None)

In [ ]:
dataset[[1,2,3]]

In [ ]:
for x in loader:
    print(x)

In [ ]:
# Array with random indices
idx_orig = np.array([1, 2, 9, 3, 4, 8, 5])
idx = np.zeros_like(idx_orig)
idx[:] = idx_orig[:]

# sort_idx is the array that induces an order on idx
sort_idx = np.argsort(idx)
print(sort_idx, idx[sort_idx])

# Applying argsort to sort_idx gives us an array that allows us to re-construct the original shuffling
sort_idx2 = np.argsort(sort_idx)
print(idx[sort_idx[sort_idx2]])

idx[sort_idx[sort_idx2]] == idx_orig

In [ ]:
idx[sort_idx[sort_idx2]]

In [ ]:
idx_orig.tolist()

In [ ]:
foo = np.zeros((2000, 1))
foo[[1,2,3]]

In [ ]:
pred_list = ["fs07_trf", "q95_trf"]

out_data = np.zeros([len(idx), len(pred_list)])

with h5py.File("/home/rkube/tmp/tmp2s2_k1aj/ty70d3n_.h5", "r") as fp:
    sort_idx = np.argsort(idx)
    for i, pred in enumerate(pred_list):
        tb = fp[f"/transformed/{pred}"]["timebase"][2]
        fetch = fp[f"/transformed/{pred}"]["signal_data"][idx[sort_idx].tolist(), :]
        #fetch = fp[f"/transformed/{pred}"]["signal_data"][idx[sort_idx]][:,0:1]
        print(fetch.shape, type(fetch))
#         print(fetch.shape)
#         out_data[:, i] = fetch[sort_idx2, 0:1]
#         print(fetch[sort_idx2])
        
print(out_data)

In [ ]:
with h5py.File("/home/rkube/tmp/tmp2s2_k1aj/ty70d3n_.h5", "r") as fp:
    sort_idx = np.argsort(idx)
    for pred in ["fs07_trf", "q95_trf"]:
        data_list = []
        for i in idx:
            data_list.append(fp[f"/transformed/{pred}"]["signal_data"][i][0])

        print(data_list)

In [ ]:
torch.tensor([1,2,3]).tolist()